## Preprocessing Avancé (Approche Hybride)

**Objectif :** Préparer le dataset final pour la modélisation.
Nous allons fusionner deux visions :
1. **La vision Macro (Externe) :** Prédire le volume global de patients en fonction du calendrier (Saisonnalité, Semaine).
2. **La vision Micro (Interne) :** Enrichir cette prédiction avec les contraintes logistiques de l'hôpital (Lits, Personnel, Pannes) et l'inertie du système (Historique).

**Méthodologie :**
* Agrégation des données au niveau **Journalier** (Granularité de pilotage de l'hôpital).
* Création de variables de **"Lag"** (Mémoire du système).
* Intégration des **Ressources Humaines et Matérielles**.

In [1]:
import pandas as pd
import numpy as np
import os

print("1. Chargement des données brutes validées...")

# On charge les données (Adapter les chemins si besoin)
try:
    df_p = pd.read_csv("../data/raw/patients.csv", parse_dates=["date_et_heure_admission"])
    df_rh = pd.read_csv("../data/raw/personnel.csv", parse_dates=["date_heure_prise_poste"])
    df_mat = pd.read_csv("../data/raw/materiel.csv", parse_dates=["date_heure_inventaire"])
    print("✅ Données chargées avec succès.")
except FileNotFoundError:
    print("Erreur : Fichiers introuvables. Vérifiez le dossier '../data/raw/'.")

# Nettoyage de base (validé en EDA)
df_p = df_p[(df_p['age'] >= 0) & (df_p['age'] <= 110)]

1. Chargement des données brutes validées...
✅ Données chargées avec succès.


### 1. Création de la Cible (Target) : Le Flux Journalier
Pour piloter l'hôpital, nous avons besoin de savoir **combien** de patients vont arriver demain.
Nous transformons donc notre liste de patients individuels en une série temporelle : **Nombre d'admissions par jour**.

In [3]:
print("Transformation en série temporelle journalière...")

# Création de la colonne date courte (YYYY-MM-DD)
df_p['date'] = df_p['date_et_heure_admission'].dt.date

# Comptage : 1 ligne = 1 jour
daily_flux = df_p.groupby('date').size().reset_index(name='nb_patients')

# Conversion en datetime pour les manipulations suivantes
daily_flux['date'] = pd.to_datetime(daily_flux['date'])

print(f"Nous avons une série historique de {len(daily_flux)} jours.")
daily_flux.head(10)

Transformation en série temporelle journalière...
Nous avons une série historique de 2922 jours.


,date,nb_patients
0,2018-01-01,416
1,2018-01-02,344
2,2018-01-03,360
3,2018-01-04,334
4,2018-01-05,333
5,2018-01-06,340
6,2018-01-07,305
7,2018-01-08,412
8,2018-01-09,338
9,2018-01-10,343


### 2. Feature Engineering : Les Cycles Temporels
L'hôpital est une horloge. La demande est dictée par deux rythmes majeurs :
1. **Le rythme Social (Hebdomadaire) :** Le pic du Lundi vs le calme du Dimanche.
2. **Le rythme Climatique (Saisonnier) :** L'impact de l'hiver (Grippe, Bronchiolite).

Nous créons ici les variables que le modèle utilisera pour comprendre ces cycles.

In [4]:
print("Création des variables calendaires...")

# 1. Jour de la semaine (0=Lundi, 6=Dimanche)
daily_flux['jour_semaine'] = daily_flux['date'].dt.weekday

# 2. Est-ce le week-end ? (Samedi ou Dimanche)
daily_flux['weekend'] = daily_flux['jour_semaine'].isin([5, 6]).astype(int)

# 3. Le mois (1-12)
daily_flux['mois'] = daily_flux['date'].dt.month

# 4. Est-ce l'Hiver ? (Décembre, Janvier, Février)
daily_flux['hiver'] = daily_flux['mois'].isin([12, 1, 2]).astype(int)

print("Contexte temporel intégré.")

Création des variables calendaires...
Contexte temporel intégré.


### 3. Feature Engineering : La Mémoire du Système (Lag Features)
C'est ici que nous ajoutons de l'intelligence "Interne".
La saturation a une **inertie**. Si l'hôpital a reçu 400 patients hier, il est probable que la tension se ressente encore aujourd'hui (lits non libérés).
Nous donnons au modèle l'information de la veille (`J-1`).

In [5]:
print("Calcul de l'inertie (Lag Features)...")

# On décale la colonne 'nb_patients' d'un cran vers le bas
daily_flux['patients_hier'] = daily_flux['nb_patients'].shift(1)

# On remplit le tout premier jour (qui n'a pas de "hier") par la moyenne globale
daily_flux['patients_hier'] = daily_flux['patients_hier'].fillna(daily_flux['nb_patients'].mean())

daily_flux[['date', 'nb_patients', 'patients_hier']].head(10)

Calcul de l'inertie (Lag Features)...


,date,nb_patients,patients_hier
0,2018-01-01,416,283.521903
1,2018-01-02,344,416.000000
2,2018-01-03,360,344.000000
3,2018-01-04,334,360.000000
4,2018-01-05,333,334.000000
5,2018-01-06,340,333.000000
6,2018-01-07,305,340.000000
7,2018-01-08,412,305.000000
8,2018-01-09,338,412.000000
9,2018-01-10,343,338.000000


## 4. Enrichissement Logistique : RH et Matériel
C'est la valeur ajoutée unique de ce projet. Contrairement à une approche purement statistique, nous intégrons la **capacité réelle** de l'hôpital.
* **RH :** Nous calculons la force de frappe journalière (Total des soignants présents).
* **Matériel :** Nous calculons le nombre total de lits disponibles et détectons les jours avec pannes critiques.

In [6]:
print("🏥 Agrégation des ressources hospitalières...")

# --- A. Préparation RH ---
df_rh['date'] = pd.to_datetime(df_rh['date_heure_prise_poste'].dt.date)
# On somme le staff présent sur tout l'hôpital pour avoir la "Force de Frappe" du jour
rh_agg = df_rh.groupby('date').agg({
    'effectif_present': 'sum', 
    'taux_absenteisme': 'mean'
}).reset_index()

# --- B. Préparation Matériel ---
df_mat['date'] = pd.to_datetime(df_mat['date_heure_inventaire'].dt.date)
# On détecte s'il y a eu AU MOINS une panne critique ce jour-là n'importe où
mat_agg = df_mat.groupby('date').agg({
    'nbre_lits_dispos': 'sum', # Total des lits libres dans l'hôpital
    'equipements_disponibles': lambda x: 1 if "Indisponible" in str(x.values) else 0
}).reset_index().rename(columns={'equipements_disponibles': 'panne_materiel'})

# --- C. Fusion Totale (Merge) ---
df_final = pd.merge(daily_flux, rh_agg, on='date', how='left')
df_final = pd.merge(df_final, mat_agg, on='date', how='left')

# Remplissage des éventuels trous (si un jour n'a pas de donnée RH, on met 0 ou la moyenne)
df_final = df_final.fillna(0)

print(f"✅ Fusion terminée. Dataset final : {df_final.shape}")
df_final.head(3)

🏥 Agrégation des ressources hospitalières...
✅ Fusion terminée. Dataset final : (2922, 11)


,date,nb_patients,jour_semaine,weekend,mois,hiver,patients_hier,effectif_present,taux_absenteisme,nbre_lits_dispos,panne_materiel
0,2018-01-01,416,0,0,1,1,283.521903,299,0.234231,146,0
1,2018-01-02,344,1,0,1,1,416.000000,306,0.221154,157,1
2,2018-01-03,360,2,0,1,1,344.000000,306,0.219231,116,0


## 5. Sauvegarde
Le dataset est maintenant "Hybride". Il contient à la fois la demande (Patients), le temps (Calendrier) et l'offre (RH/Lits). Il est prêt pour le Machine Learning.

In [7]:
os.makedirs("../data/processed", exist_ok=True)
output_path = "../data/processed/train_data_hybride.csv"
df_final.to_csv(output_path, index=False)

print(f"💾 Fichier sauvegardé : {output_path}")
print("🚀 Prêt pour le notebook de Machine Learning !")

💾 Fichier sauvegardé : ../data/processed/train_data_hybride.csv
🚀 Prêt pour le notebook de Machine Learning !
